In [1]:
import pandas as pd
from tqdm.auto import tqdm
import ast

tqdm.pandas()

In [2]:
df = pd.read_csv("./data/credits.csv")

In [3]:
df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
def move_movie_id_in_jsons(row: pd.Series) -> pd.Series:
    cast, crew, movie_id = (
        ast.literal_eval(row["cast"])[:5],
        ast.literal_eval(row["crew"])[:3],
        row["id"],
    )

    for c in cast:
        c["movie_id"] = movie_id

    for c in crew:
        c["movie_id"] = movie_id

    return pd.Series([cast, crew], index=["cast", "crew"])

In [5]:
df = df.progress_apply(move_movie_id_in_jsons, axis=1)

  0%|          | 0/45476 [00:00<?, ?it/s]

In [6]:
cast, crew = df["cast"], df["crew"]

In [7]:
cast = cast[cast.map(lambda x: len(x) > 0)].reset_index(drop=True)
crew = crew[crew.map(lambda x: len(x) > 0)].reset_index(drop=True)

In [8]:
cast_df = pd.DataFrame(cast.explode().reset_index(drop=True).values.tolist())

In [9]:
crew_df = pd.DataFrame(crew.explode().reset_index(drop=True).values.tolist())

In [10]:
cast_info = (
    cast_df[["id", "name", "gender"]]
    .drop_duplicates(subset=["id"])
    .reset_index(drop=True)
)
cast_movie_relationship = cast_df[["id", "movie_id", "character"]].rename(
    columns={"id": "actor_id"}
)

In [11]:
crew_info = (
    crew_df[["id", "name", "gender"]]
    .drop_duplicates(subset=["id"])
    .reset_index(drop=True)
)
crew_movie_relationship = crew_df[["id", "movie_id", "department"]].rename(
    columns={"id": "actor_id"}
)
crew_movie_relationship["department"] = crew_movie_relationship[
    "department"
].str.upper()

In [14]:
cast_info.to_csv("./data/cast_info.csv", index=False)
cast_movie_relationship.to_csv("./data/cast_movie_relationship.csv", index=False)
crew_info.to_csv("./data/crew_info.csv", index=False)
crew_movie_relationship.to_csv("./data/crew_movie_relationship.csv", index=False)